# Image registration metrics for the unstain2stain images:
-Mutual Information
-Target Registration Error (SITK)

In [3]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import cv2
from matplotlib import pyplot as plt
from glob import glob

### Mutual Information (MI):

In [22]:
# from skin_2D_workflow\image_registration_evaluation to calculate mutual information:
# Function to calculate MI:
def calculate_mutual_information(img_path_1,img_path_2): #order doesn't matter, MI(A,B) = MI(B,A)
    img1 = np.array(cv2.imread(img_path_1))
    img2 = np.array(cv2.imread(img_path_2))
    img1_g = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
    img2_g = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
    hist2d, x_edges, y_edges = np.histogram2d(img1_g.ravel(),img2_g.ravel(),bins=20)
    pxy = hist2d / float(np.sum(hist2d))
    px = np.sum(pxy, axis=1)  # marginal x over y
    py = np.sum(pxy, axis=0)  # marginal y over x
    px_py = px[:, None] * py[None, :]  # broadcast to multiply marginals
    # now we can do the calculation using the pxy, px_py 2D arrays
    nonzeros = pxy > 0  # filer out the zero values
    mi = np.sum(pxy[nonzeros] * np.log(pxy[nonzeros] / px_py[nonzeros]))
    return mi

In [23]:
img_path_1 = glob(os.path.join(r"\\shelter\Kyu\unstain2stain\biomax_images\registrated_images","**","**.png"),recursive=True) #unstained, 174 images same order
img_path_2 = glob(os.path.join(r"\\shelter\Kyu\unstain2stain\biomax_images\stained\padded_images","**","**.png"),recursive=True) #stained, 174 images same order

In [24]:
# MI for each pair of image:
mi_ra = []
for idx in range(0,len(img_path_1)):
    img1 = img_path_1[idx]
    img2 = img_path_2[idx]
    mi = calculate_mutual_information(img1,img2)
    mi_ra.append(mi)
print(mi_ra)

[0.06960824587047423, 0.11743108445857318, 0.14049127955630492, 0.04516067106990143, 0.07190700019514393, 0.1026032203130491, 0.03944753233642199, 0.12487647436827311, 0.08166988417207607, 0.07407601452823742, 0.08675877265733534, 0.026672960834480484, 0.03882193563466624, 0.08350645432596476, 0.047932620308268954, 0.1557985326446766, 0.10315432003633795, 0.037578114212290056, 0.06191071062390437, 0.04281327678146191, 0.05740118990975115, 0.11081469698863479, 0.12363178346856792, 0.1531894553591467, 0.06883738548188653, 0.039376671331586136, 0.1283598570718828, 0.08570480086665128, 0.06703886482233695, 0.06059291555478167, 0.07624198946936489, 0.03936230216049576, 0.03796931610627313, 0.08174288018862663, 0.062358578765578675, 0.10845610990246536, 0.047642168106918764, 0.15492445500665886, 0.049275052229929325, 0.009416651217994598, 0.08069333541634434, 0.07923558485749368, 0.01876567784599072, 0.046131298466690014, 0.05467380562964635, 0.05526217760977068, 0.08458265097445196, 0.04745

b/c stained vs unstained image different modality, mutual information doesn't perform as well, as seen in the low scores above for all 174 images.

### Target Registration Error (TRE):